给你一个整数数组 nums ，找到其中最长严格递增子序列的长度。

子序列 是由数组派生而来的序列，删除（或不删除）数组中的元素而不改变其余元素的顺序。例如，[3,6,2,7] 是数组 [0,3,1,6,2,2,7] 的子序列。

示例 1：
输入：nums = [10,9,2,5,3,7,101,18]
输出：4
解释：最长递增子序列是 [2,3,7,101]，因此长度为 4 。

示例 2：
输入：nums = [0,1,0,3,2,3]
输出：4

示例 3：
输入：nums = [7,7,7,7,7,7,7]
输出：1

方法1：动态规划
主问题：dp[i] 表示以nums[i]结尾（即 [0, i] 范围）的最长递增子序列长度
子问题：从i前面找到所有j （即 j 属于 [0, i-1]）且满足nums[j]<nums[i]，那么意味着i可以接在j后面
状态转移：得到 dp[i] = max(dp[j1] + 1, dp[j2] + 1, dp[j3] + 1 ......)

In [ ]:
# 方法1：动态规划
class Solution:

    def lengthOfLIS(self, nums: List[int]) -> int:
        n = len(nums)
        f = [1] * n
        for i, x in enumerate(nums):
            for j in range(i):
                if nums[j] < x:
                    f[i] = max(f[i], 1 + f[j])
        return max(f)

        # @lru_cache(None)
        # def dfs(i):  # 以nums[i]结尾的 LIS
        #     ans = 1
        #     for j in range(i):
        #         if nums[j] < nums[i]:
        #             ans = max(ans, 1 + dfs(j))
        #     return ans
        # return max(dfs(i) for i in range(n))

方法2：贪心+二分
假设一个数组为 [1,3,5] 现在末尾增加一个数字4, 数字4应该和原来数组构成LIS是 [1,3,4]还是[1,4,5]？ 他们长度都是3
虽然两个LIS长度都为3，但是我们贪心选择结尾更小的[1,3,4]因为这样可以保证后面再添加一个数更有可能接在后面，
比如现在末尾增加一个数字5，前面选择[1,3,4]可以构成[1,3,4,5]长度为4的LIS，如果前面选择[1,4,5]就无法构成长度>3的LIS了
因此得出一个结论，我们贪心地将LIS结尾越小越好，方便后面拼接
因此如果我们要添加x，我们可以利用二分查找，找到第一个>=x的位置，进行更换，如果找不到，那就意味着原数组全部<x，可以直接加在末尾

In [ ]:
# 方法2：贪心+二分
class Solution:

    def lengthOfLIS(self, nums: List[int]) -> int:
        # 核心思想 尽量让小数接在结尾 方便使得LIS变长
        lst = []  # 最长递增子序列 idx+1 长度对应的最大值
        ans = 0
        for x in nums:
            i = bisect_left(lst, x)  # >=x的第一个坐标
            if i == len(lst):  # 全<x 末尾增加
                lst.append(x)
                ans += 1
            else:
                lst[i] = x  # 更新lst[i]是x可以放的位置
        return ans

方法3：线段树加速动态规划
和方法1不同，将值域作为动态规划参数就行dp
主问题：dp[i] 表示以 数字i这个值 结尾的LIS长度
子问题：可以从[...,i-1]前面所有子问题进行转移，取最大
状态转移：dp[i] = max(dp[mn], dp[mn - 1] ... dp[i - 1]) 而连续范围取最值可以利用线段树快速求得

In [ ]:
# 方法3：线段树加速动态规划
class Solution:

    def lengthOfLIS(self, nums: List[int]) -> int:
        # dp[i] 以 i数 结尾的LIS
        # dp[i] = max(dp[j])  mn <= j <= i - 1  从前面转移
        # basecase dp[mn] = 1

        mn, mx = min(nums), max(nums)
        vals = [0] * (4 * (mx - mn + 1))

        def update(o, l, r, idx, val):
            if l == r:
                vals[o] = val
                return
            m = (l + r) // 2
            if m >= idx: update(o * 2, l, m, idx, val)
            else: update(o * 2 + 1, m + 1, r, idx, val)
            vals[o] = max(vals[o * 2], vals[o * 2 + 1])

        def query(o, l, r, L, R):  # [L, R]
            if L <= l and r <= R:
                return vals[o]
            m = (l + r) // 2
            ans = -inf
            if m >= L: ans = max(ans, query(o * 2, l, m, L, R))
            if m + 1 <= R: ans = max(ans, query(o * 2 + 1, m + 1, r, L, R))
            return ans

        for x in nums:
            if x == mn:
                update(1, mn, mx, x, 1)
            else:
                update(1, mn, mx, x, 1 + query(1, mn, mx, mn, x - 1))
        return vals[1]


In [ ]:
# lc926 最长不下降子序列
class Solution:
    def minFlipsMonoIncr(self, s: str) -> int:
        # LIS 最长不下降子序列
        lst = []
        lgh = 0
        for x in s:
            l = bisect_right(lst, x) # >
            if l == lgh: # 全 <= 可以增加长度了  因为是不下降所以在 <= 添加
                lst.append(x)
                lgh += 1
            else:
                lst[l] = x
        return len(s) - lgh